In [1]:
# Character level lyrics generation using RNNs (LSTM)
import sys, os, random, string
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import string

import CharlyricsDataset
import glob

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# interactive mode
plt.ion()

from pathlib import Path
from config import config
import utils

%load_ext autoreload
%autoreload 2

In [2]:
train_dataset = CharlyricsDataset.CharLyricsDataset(config.DATA.LYRICS)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.TRAIN.BATCH_SIZE,
    shuffle=False,
    drop_last=True,
    num_workers=1
)

In [3]:
for _, batch in enumerate(train_loader):
    print(batch)
    print()
    break

tensor([[38, 10, 23, 94, 32, 14, 94, 15, 24, 27, 16, 14, 29, 94, 10, 11, 24, 30,
         29, 94, 29, 17, 14, 94, 29, 17, 18, 23, 16, 28, 94, 44, 94, 28, 10, 18,
         13, 94, 32, 17, 14, 23, 94, 44, 94, 32, 10, 28, 94, 13, 27, 30, 23, 20,
         75, 75, 75, 96, 44, 94, 13, 18, 13, 23, 68, 29, 94, 22, 14, 10, 23, 94,
         29, 24, 94, 12, 10, 21, 21, 94, 34, 24, 30, 94, 29, 17, 10, 29, 96, 44,
         94, 12, 10, 23, 68, 29, 94, 27, 14, 22],
        [14, 22, 11, 14, 27, 94, 32, 17, 10, 29, 94, 32, 10, 28, 94, 28, 10, 18,
         13, 96, 50, 27, 94, 32, 17, 10, 29, 94, 34, 24, 30, 94, 29, 17, 27, 14,
         32, 94, 10, 29, 94, 22, 14, 94, 51, 21, 14, 10, 28, 14, 94, 29, 14, 21,
         21, 94, 22, 14, 96, 51, 21, 14, 10, 28, 14, 94, 29, 14, 21, 21, 94, 22,
         14, 94, 32, 17, 34, 96, 48, 34, 94, 12, 10, 27, 94, 18, 28, 94, 18, 23,
         94, 29, 17, 14, 94, 15, 27, 24, 23, 29],
        [94, 34, 10, 27, 13, 96, 36, 23, 13, 94, 44, 94, 10, 22, 94, 28, 21, 14,
         